In [2]:
import numpy as np
import pandas as pd
import time

In [ ]:
prod_filepath = "./data/production_data.csv"
df_prod = pd.read_csv(prod_filepath)

alias_filepath = "./data/operator_aliasing.csv"
df_alias = pd.read_csv(alias_filepath)

wells_filepath = "./data/wellsData.csv"
df_wells = pd.read_csv(wells_filepath)

Adjusting API by adding a leading zero to APIs starting with '5'

In [ ]:
start = time.time()
df_prod.API=df_prod.API.astype(str).apply(lambda value:value.zfill(14))

Add Operator Alias

In [ ]:
filter_col = df_prod.columns
filter_col = np.append(filter_col, ['Alias'])

df_wells.API=df_wells['API'].astype(str)

wells_alias_df = df_alias[['Name', 'Alias']].merge(df_wells[['operatorNameIHS', 'API']], left_on='Name', right_on='operatorNameIHS', how='outer')
new_df = (df_prod.merge(wells_alias_df[['operatorNameIHS', 'Alias', 'API']], on='API', how='left'))

new_df.Alias.fillna(new_df.operatorNameIHS, inplace=True)

Remove Pre-Peak Months

In [ ]:
# filtering out post-peak records with 0 oil production
tmp_df = new_df[filter_col][new_df['Liquid']!=0]

# getting peak records
prod_grouped = tmp_df.groupby(['API'], sort=False)
peak_records = tmp_df[prod_grouped['Liquid'].transform(max) == tmp_df['Liquid']]

# filtering out pre-peak months
peak_records = peak_records.rename(columns={"Month": "PeakMonth", "Year": "PeakYear"})
tmp_df = pd.merge(tmp_df, peak_records[["API", "PeakMonth", "PeakYear"]], on='API', how='left')
post_peak = tmp_df[np.logical_or((tmp_df['Year'] > tmp_df['PeakYear']),np.logical_and((tmp_df['Year']==tmp_df['PeakYear']),(tmp_df['Month']>tmp_df['PeakMonth'])))].sort_values(['Year','Month'])[filter_col]

Add a Month Index Column

In [ ]:
post_peak['MonthIdx'] = post_peak.groupby(['API'], sort=False).cumcount()+1
end = time.time()
tt = end - start
print(tt)

In [ ]:
# tmp_df.to_csv('data/out.csv')